In [3]:
import pandas as pd
import numpy as np

In [4]:
df = message = pd.read_csv('SMSSpamCollection.txt', sep='\t',
                           names=["label", "message"])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:

X=list(df['message'])
y=list(df['label'])

In [6]:
y=list(pd.get_dummies(y,drop_first=True)['spam'])

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [8]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [9]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [10]:
import tensorflow as tf

In [11]:
test_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [12]:
tf_train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    list(y_train)
)).batch(16)

tf_validation_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    list(y_test)
)).batch(16)

In [13]:
from transformers import TFAutoModelForSequenceClassification, create_optimizer

# Define optimizer and learning rate schedule
num_epochs = 2
batch_size = 16
total_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps
)

# Loading the model
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

#Loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

# Fine tuned with initial parameters

In [14]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs
)


Epoch 1/2

279/279 [==============================] - 2148s 8s/step - loss: 0.0958 - val_loss: 0.0248
Epoch 2/2
279/279 [==============================] - 2183s 8s/step - loss: 0.0238 - val_loss: 0.0235


In [15]:
model.save_pretrained('fine_tuned_model')

In [16]:
model.get_weights()

[array([[-0.01664949, -0.06661227, -0.01632868, ..., -0.01999032,
         -0.05139988, -0.0263568 ],
        [-0.01319846, -0.06733431, -0.01605646, ..., -0.0226614 ,
         -0.05537301, -0.02600443],
        [-0.01759106, -0.07094341, -0.01443494, ..., -0.02457913,
         -0.05956192, -0.0231829 ],
        ...,
        [-0.0231029 , -0.05878259, -0.01048967, ..., -0.01945743,
         -0.02615411, -0.02118432],
        [-0.0490171 , -0.05614787, -0.00465348, ..., -0.01065376,
         -0.01797333, -0.02187675],
        [-0.00646111, -0.0914881 , -0.00254872, ..., -0.01505679,
         -0.05040044,  0.04597744]], dtype=float32),
 array([[ 0.01796749, -0.02545969, -0.03723966, ..., -0.00010971,
          0.00089723,  0.01581317],
        [ 0.0074049 ,  0.00263125, -0.01934073, ...,  0.02918059,
          0.03010739, -0.00552881],
        [-0.01159471, -0.00192955, -0.01147501, ...,  0.01494822,
          0.01907524, -0.00745037],
        ...,
        [ 0.01741793,  0.00349027, -0.0

In [17]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

original_model = TFAutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased')
fine_tuned_model = model
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [21]:
import evaluate

In [22]:
# Metrics

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

In [23]:
# Sample test data (replace with your actual test dataset)
texts = ["Free entry to a contest!", "Hi, can we meet tomorrow?", "Congratulations! You've won a prize!", "Let's discuss the project plan."]
labels = [1, 0, 1, 0]  # Assuming 1 = Spam, 0 = Ham

# Tokenize and prepare data
encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="tf")
input_ids = encodings['input_ids']
attention_mask = encodings['attention_mask']

In [24]:
# Function to get predictions and evaluate metrics using Hugging Face's evaluate library
def evaluate_model(model):
    predictions = model(input_ids, attention_mask=attention_mask).logits
    predicted_labels = tf.argmax(predictions, axis=1).numpy()

    # Computing the metrics
    accuracy_result = accuracy.compute(references=labels, predictions=predicted_labels)
    f1_result = f1.compute(references=labels, predictions=predicted_labels, average="binary")
    precision_result = precision.compute(references=labels, predictions=predicted_labels, average="binary")
    recall_result = recall.compute(references=labels, predictions=predicted_labels, average="binary")

    return {
        "Accuracy": accuracy_result["accuracy"],
        "F1 Score": f1_result["f1"],
        "Precision": precision_result["precision"],
        "Recall": recall_result["recall"]
    }

In [24]:
# Evaluate both models
original_metrics = evaluate_model(original_model)
fine_tuned_metrics = evaluate_model(fine_tuned_model)

In [25]:
# Display the results
print("Original Model Metrics:")
print(original_metrics)
print("\nFine-Tuned Model Metrics:")
print(fine_tuned_metrics)

Original Model Metrics:
{'Accuracy': 0.5, 'F1 Score': 0.6666666666666666, 'Precision': 0.5, 'Recall': 1.0}

Fine-Tuned Model Metrics:
{'Accuracy': 0.75, 'F1 Score': 0.6666666666666666, 'Precision': 1.0, 'Recall': 0.5}


# Fine tuning with new parameters

In [13]:
from transformers import TFAutoModelForSequenceClassification, create_optimizer
import tensorflow as tf

# Adjusted hyperparameters
num_epochs = 3  # Increase the number of epochs
batch_size = 16  # Batch size
initial_learning_rate = 3e-5  # Initial learning rate
end_learning_rate = 1e-6  # Minimum learning rate at the end of training
total_train_steps = len(tf_train_dataset) * num_epochs  # Total training steps
decay_steps = total_train_steps  # Decay steps equal to total training steps
power = 1.0  # Linear decay

In [14]:
# Define the Polynomial Decay learning rate schedule
learning_rate_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate,
    decay_steps=decay_steps,
    end_learning_rate=end_learning_rate,
    power=power
)

In [17]:
# Create the optimizer with the learning rate schedule
optimizer, schedule = create_optimizer(
    init_lr=initial_learning_rate,  # Initial learning rate for the optimizer
    num_warmup_steps=0,  # No warmup steps
    num_train_steps=total_train_steps,  # Total training steps
    power=power  # Power for polynomial decay
)

In [18]:
# Loading the model
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

#Loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [19]:
Updated_Model = model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,  # Updated epoch count
    batch_size=batch_size
)

Epoch 1/3

279/279 [==============================] - 1895s 7s/step - loss: 0.0845 - val_loss: 0.0229
Epoch 2/3
279/279 [==============================] - 1947s 7s/step - loss: 0.0218 - val_loss: 0.0194
Epoch 3/3
279/279 [==============================] - 2169s 8s/step - loss: 0.0098 - val_loss: 0.0219


In [26]:
Updated_Model_Metrics = evaluate_model(Updated_Model)

TypeError: 'History' object is not callable